In [1]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core

!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 17.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15837, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 15837 (delta 35), reused 14 (delta 14), pack-reused 15772 (from 2)
Receiving objects: 100% (15837/15837), 6.41 MiB | 19.36 MiB/s, done.
Resolving deltas: 100% (11533/11533), done.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your 

In [2]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
torch:  2.6 ; cuda:  cu124
detectron2: 0.6


In [3]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [4]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/drive/MyDrive/ESC499/Notehead_Detection/data/train/train.json", "/content/drive/MyDrive/ESC499/Notehead_Detection/data/train")
register_coco_instances("my_dataset_val", {}, "/content/drive/MyDrive/ESC499/Notehead_Detection/data/val/val.json", "/content/drive/MyDrive/ESC499/Notehead_Detection/data/val")

train_metadata = MetadataCatalog.get("my_dataset_train")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train")
val_metadata = MetadataCatalog.get("my_dataset_val")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val")


from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

class TrainerWithEvaluator(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Defines the evaluator for validation.
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)


cfg = get_cfg()
cfg.OUTPUT_DIR = "/content/drive/MyDrive/ESC499/Notehead_Detection/model_NH"
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.MASK_ON = False  # no segmentation field

cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.TEST.EVAL_PERIOD = 500
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0002
cfg.SOLVER.MAX_ITER = 3000
cfg.SOLVER.STEPS = [2000, 2500]
cfg.SOLVER.GAMMA = 0.5
# cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = TrainerWithEvaluator(cfg)         # Create an instance of of DefaultTrainer with the given congiguration
trainer.resume_or_load(resume=False)        # Load a pretrained model if available (resume training) or start training from scratch if no pretrained model is available


[03/23 19:08:49 d2.data.datasets.coco]: Loading /content/drive/MyDrive/ESC499/Notehead_Detection/data/train/train.json takes 3.65 seconds.
[03/23 19:08:49 d2.data.datasets.coco]: Loaded 272 images in COCO format from /content/drive/MyDrive/ESC499/Notehead_Detection/data/train/train.json
[03/23 19:08:51 d2.data.datasets.coco]: Loading /content/drive/MyDrive/ESC499/Notehead_Detection/data/val/val.json takes 1.22 seconds.
[03/23 19:08:51 d2.data.datasets.coco]: Loaded 70 images in COCO format from /content/drive/MyDrive/ESC499/Notehead_Detection/data/val/val.json
[03/23 19:08:52 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 25

model_final_280758.pkl: 167MB [00:00, 252MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


In [5]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9
predictor = DefaultPredictor(cfg)

[03/23 19:08:59 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/ESC499/Notehead_Detection/model_NH/model_final.pth ...


In [ ]:
input_images_directory = "/content/drive/MyDrive/ESC499/Notehead_Detection/data/test"
output_directory = "/content/drive/MyDrive/ESC499/Notehead_Detection/data/test_results"
os.makedirs(output_directory, exist_ok=True)

for image_filename in os.listdir(input_images_directory):
    image_path = os.path.join(input_images_directory, image_filename)
    new_im = cv2.imread(image_path)

    outputs = predictor(new_im)  # Format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format

    # Use Visualizer to draw the predictions on the image
    v = Visualizer(new_im[:, :, ::-1], metadata=train_metadata)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    # Create the output filename with _result extension
    result_filename = os.path.splitext(image_filename)[0] + "_result.png"
    output_path = os.path.join(output_directory, result_filename)
    # Save the segmented image
    cv2.imwrite(output_path, out.get_image()[:, :, ::-1])

print("Segmentation of all images completed.")

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Segmentation of all images completed.


In [6]:
import csv
from detectron2.utils.visualizer import GenericMask

input_images_directory = "/content/drive/MyDrive/ESC499/Notehead_Detection/data/test"
output_csv_path = "/content/drive/MyDrive/ESC499/Notehead_Detection/data/notehead_detection.csv"

# Open the CSV file
with open(output_csv_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write the header row
    csvwriter.writerow(["File Name", "Class Name", "Object Number", "Area", "Centroid", "BoundingBox"])

    # Iterate over images in the input directory
    for image_filename in os.listdir(input_images_directory):
        image_path = os.path.join(input_images_directory, image_filename)
        new_im = cv2.imread(image_path)

        # Perform prediction
        outputs = predictor(new_im)
        instances = outputs["instances"].to("cpu")

        # Extract information from predictions
        class_labels = instances.pred_classes.numpy()  # Class IDs
        boxes = instances.pred_boxes.tensor.numpy()  # Bounding boxes

        for i, box in enumerate(boxes):
            # Extract bounding box coordinates
            x_min, y_min, x_max, y_max = box

            # Compute bounding box area
            bbox_area = round(float((x_max - x_min) * (y_max - y_min)), 4)

            # Compute centroid
            centroid_x = float((x_min + x_max) / 2)
            centroid_y = float((y_min + y_max) / 2)
            centroid = [round(centroid_x, 3), round(centroid_y, 3)]

            # Class name
            class_name = train_metadata.thing_classes[class_labels[i]]

            # Bounding box with rounded values
            bbox_rounded = [round(float(coord), 3) for coord in box]

            # Write to CSV
            csvwriter.writerow([image_filename, class_name, i + 1, bbox_area, centroid, bbox_rounded])

print(f"Object detection results saved to {output_csv_path}")

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Object detection results saved to /content/drive/MyDrive/ESC499/Notehead_Detection/data/notehead_detection.csv
